**EM Algorithm** (Expectation - Maximization)

We're going to implement the EM algorithm for a mixture of Bernoullis

The Expectation-Maximization algo. is used in sk-learn, for instance in GMMs: http://scikit-learn.org/stable/modules/mixture.html#estimation-algorithm-expectation-maximization

Additional notes are available here: https://allauzen.github.io/articles/MixturesAndEM/


## Think for a bit

Is EM a supervised or unsupervised learning algorithm ? 

What kind of data set is MNIST ? 

Then, what are we going to do ?  How is this called ?




## Implement the EM algo. for a mixture of Bernoulli (laws)
- The cluster number K should be an argument of the function
- A maximum number of iterations, *MaxIt*, should act as stopping condition
- During the E step, compute and *store* the log-likelihood of the data, so as to monitor its evolution along iterations (~epochs)

Apply the algorithm on MNIST:
- try out K=5,10,15
- Visualize the images coresponding to each cluster's paraemeters. Would that be as straightforward in a Gaussian model (visualizing all the model's parameters?)

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [4]:
## we load the whole data set at once, so as to manipulate it directly (using numpy arrays)
## for very large data sets, one needs to read on-the-fly (at least at production time, not at the debugging stage)

# ## Load the dataset -- (python2)
# import cPickle, gzip
# with gzip.open('./tp1-mnist.pkl.gz','rb') as f :
#     train_set, valid_set, test_set = cPickle.load(f)

### Load the dataset -- (python3)
import pickle, gzip
with gzip.open('./mnist.pkl.gz','rb') as f :
    u = pickle._Unpickler( f )
    u.encoding = 'latin1'
    train_set, valid_set, test_set = u.load()

## split train an dtest data, to avoid inadvertently cheating.
unlabelled_dataset = train_set[0].copy()
labels_for_final_accuracy_measurement = train_set[1].copy()
del train_set

## these would be useful, for now we don't use them
del valid_set  
del test_set

In [5]:
unlabelled_dataset

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
unlabelled_dataset.shape

(50000, 784)

In [8]:
## binarize so as to go into Bernoulli-space
threshold = 0.3
for i in range(unlabelled_dataset.shape[0]): 
    unlabelled_dataset[i,:]= 1.0*(unlabelled_dataset[i]>threshold)
## (unlabelled_dataset[i]>threshold) is now an array of Booleans

In [17]:
## inspect the new values "by hand", to check.
unlabelled_dataset[0][300:400] ## reading 
## we do NOT convert to Integers, instead we stay in floats. Because we'll compute averages later (thus, divide..)

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
      dtype=float32)

In [12]:
## for debugging, we may want to use a small piece of the sample
subSampling = 100  ## data size will be divided by a factor "subSampling"
## in the end, we may use all of it (set this to 1) 

## sort data to make sure all classes are equally present ##
ordre = np.argsort(labels_for_final_accuracy_measurement)
unlabelled_dataset = unlabelled_dataset[ordre]
labels_for_final_accuracy_measurement = labels_for_final_accuracy_measurement[ordre]

## do the sub-sampling
dataset = unlabelled_dataset[::subSampling].copy()
subSampledLabels = labels_for_final_accuracy_measurement[::subSampling].copy()

In [13]:
## check it:
subSampledLabels[:10000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [32]:
## when you code, start by writing on a copy of the data 
## (not the data themselves: otherwise on the second run of the jupyter cell, you'll have run twice)
K=10
MaxIt=4
dim = (dataset.shape)[1]
k=4
xi=dataset[0]
theta = np.zeros((K, dim))

## compute the proba of an image xi, 
## given its cluster k and parameters theta 
def compute_P_Xi_given_k_and_theta(xi, k, theta):
    #subSLabels=subSampledLabels.copy()
    logpi=xi*np.log(theta+10**-8)+(1-xi)*np.log(1-theta+10**-8)
    output=sum(logpi)
    return output

outputForDebug = compute_P_Xi_given_k_and_theta(xi, k, theta)

In [34]:
print(outputForDebug)

[ 9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999989e-08  9.99999989e-08  9.99999989e-08
  9.99999989e-08  9.99999

In [45]:
print(Nex,dim, classFrequencies, theta, affectations, K)

500 784 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0.07495811 0.14767547 0.12806177 ... 0.02121543 0.1792508  0.17086294]
 [0.13972571 0.08327242 0.05415162 ... 0.03734071 0.17480676 0.07727137]
 [0.03705134 0.08417405 0.1527212  ... 0.12649736 0.06986583 0.14957424]
 ...
 [0.05734223 0.12467498 0.17446599 ... 0.10768716 0.09748848 0.02795721]
 [0.11447567 0.12510867 0.13298956 ... 0.03634823 0.04807512 0.0726098 ]
 [0.11994    0.07979479 0.05041766 ... 0.08947527 0.15006547 0.10348553]] 10


In [83]:
affectations = np.random.random((K, Nex))
#for i in range(Nex):
#    affectations[:,i] / np.sum(affectations[:,i])
affectations.shape
data=dataset.shape(10, 500)

TypeError: 'tuple' object is not callable

In [44]:
print(affectations[:,1] / np.sum(affectations[:,1]))

[0.14767547 0.08327242 0.08417405 0.07443183 0.03836348 0.11875492
 0.12374938 0.12467498 0.12510867 0.07979479]


In [63]:
data=dataset
theta=affectations*dataset

ValueError: operands could not be broadcast together with shapes (10,500) (500,784) 

In [88]:
## python-note: at debug time, don't write a function, instead, write directly in the main
## when evreything works well, encapsulate the piece of code into a function for later re-use (and lisibility)
#def AlgoEM(dataset, K, MaxIt, labels_for_final_accuracy_measurement):
    

## we set the seed to a constant so as to have repeatable experiences 
## (at debug time, and when comparing hyper-parameters)
np.random.seed(42)

## we'll assume dataset[i,j] is the value of pixel j of image i.
Nex = (dataset.shape)[0] ## number of examples (= number of images)
dim = (dataset.shape)[1] ## dimension of parameters space (=number of pixels per image, in the Bernoulli case)
classFrequencies = np.zeros(K) ## denoted pi_k 
theta = np.zeros((K, dim))   ## denoted theta_{k,j}

## initialization ##
affectations = np.random.random((K, Nex)) ## denoted a_{k,i}
## normalization step (needed to make the )
for i in range(Nex):
    affectations[:,i] / np.sum(affectations[:,i])
    
## main loop ##
for iteration in range(0,MaxIt,1):
    #if iteration%10 == 0:
    #    print("iteration numero"+str(iteration))
       

    ######################
    ## step M: update of "pi_k, mu_k" (classFrequencies, theta)  ##
    for k in range(K):
        classFrequencies[k]=np.sum(affectations[k])/sum(affectations)
        
    
    theta=np.dot(affectations*dataset)
    #For     
    #    theta[k]=np.dot(sum(affectation[k])*
    

    ######################
    ## step E: update of "a_ik" (affectations) ##
    ## TODO 

    ## monitoring of the quality of te clustering ##
    #LogLikelihood = np.log(np.max(affectations, axis=0)) 
    #print(np.mean(LogLikelihood))
    ## TODO: record the monitoring into an array, to be able to later plot it

return classFrequencies[k]
##########################################################################


ValueError: setting an array element with a sequence.

In [ ]:
classFrequencies

In [ ]:
plt.plot(LogVraisemblance)

In [ ]:
## we can visualize the parameters theta, cluster by cluster
for k in range(K):
    plt.imshow(theta[k].reshape(28,28) , matplotlib.pyplot.cm.jet)
    matplotlib.pyplot.show()

TODO: after checking your code works, put it inside a function and call that function 
 (instead of running code in the *main()* directly)

In [ ]:
# affectations, theta, classFrequencies = AlgoEM(dataset, K, MaxIt)

Now, let's monitor something more interpretable than the log-likelihood... what could it be ? 

Trick: you may use the function np.argmax(), and maybe just a couple of labels from a couple of examples

**Quesiton**: would this strategy be possible for a purely unsupervised task ?

In [ ]:
## TODO: (later) 
## what , 0,1,2,...9
# prediction = np.argmax(affectations, axis=0)
# score = np.mean(prediction == labels_for_final_accuracy_measurement[:subSampling])
# print(score)

Now add this interesting metric into your function, and add some plot of it from the function as well (possibly along wth the log-likelihood plot)

Now, you may play with the hyper-parameters.

What is the best value of the main hyper-parameter ? Explain why.